In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import missingno as msno
import statsmodels.formula.api as smf
from scipy import stats


%matplotlib inline

In [ ]:
execs = pd.read_csv('../../data/raw/CSVs_Processados/execucoes.csv', parse_dates=['Data_Execucao',
                                                                                  'Data_FechamentoCenario',
                                                                                  'TimeStamp_Execucao'])
execs.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
execs.shape

In [ ]:
#Checando valores nulos da tabela
msno.matrix(execs)

In [ ]:
execs.head()

## Análise e tratamentos dos dados de sistemas

In [ ]:
#Criando dataframe para trabalhar com os sistemas
sistemas = execs[['Sistema_Principal',
                  'ID_RUN',
                  'Classificacao',
                  'Duracao_Segundos',
                  'Motivo_Execucao',
                  'Qtde_StepsATTACHMENT',
                  'Qtde_TotalSteps',
                  'Tipo_Execucao',
                  'Target']]

In [ ]:
sistemas.describe().T

Duração possui muitos casos zerados. Apenas pelo describe, podemos ver que no mínimo 50% dos tempos estão zerados. Erro na hora de inputar os dados ou foi feito fora do processo padrão?

Para os sistemas que possuem tempo zerado, porém temos vários casos em que há duração, podemos realizar uma regressão. Já para os casos que estão totalmente zerados ou próximo disso, realizar uma regressão seria inviável.

Vamos, primeiramente, analisar os nossos dados.

In [ ]:
sns.pairplot(sistemas, hue='Tipo_Execucao', hue_order='')

### 1. Comparando dados de Fast e Run

In [ ]:
qtd_execs = sistemas['Duracao_Segundos'].count() 

qtd_fast = sistemas[sistemas['Tipo_Execucao'] == 'FAST']['Duracao_Segundos'].count()
print('% Fast do total: ' + str(round((qtd_fast/qtd_execs)*100, 2)) + '%')
print('Media Fast: ' + str(sistemas[sistemas['Tipo_Execucao'] == 'FAST']['Duracao_Segundos'].mean()))
print('Std Fast: ' + str(sistemas[sistemas['Tipo_Execucao'] == 'FAST']['Duracao_Segundos'].std()))
qtd_zeros_fast = sistemas[(sistemas['Tipo_Execucao'] == 'FAST') & (sistemas['Duracao_Segundos'] == 0)]['Duracao_Segundos'].count()
total_fast = sistemas[(sistemas['Tipo_Execucao'] == 'FAST')]['Duracao_Segundos'].count()
print('% de zeros Fast: ' + str(round((qtd_zeros_fast/total_fast)*100, 2)) + '%')

print('')

qtd_run = sistemas[sistemas['Tipo_Execucao'] == 'RUN']['Duracao_Segundos'].count()
print('% Run do total: ' + str(round((qtd_run/qtd_execs)*100, 2)) + '%')
print('Media Run: ' + str(sistemas[sistemas['Tipo_Execucao'] == 'RUN']['Duracao_Segundos'].mean()))
print('Std Run: ' + str(sistemas[sistemas['Tipo_Execucao'] == 'RUN']['Duracao_Segundos'].std()))
qtd_zeros_run = sistemas[(sistemas['Tipo_Execucao'] == 'RUN') & (sistemas['Duracao_Segundos'] == 0)]['Duracao_Segundos'].count()
total_run = sistemas[(sistemas['Tipo_Execucao'] == 'RUN')]['Duracao_Segundos'].count()
print('% de zeros Run: ' + str(round((qtd_zeros_run/total_run)*100, 2)) + '%')

In [ ]:
sistemas[sistemas['Tipo_Execucao'] == 'FAST'].describe().T

In [ ]:
sistemas[sistemas['Tipo_Execucao'] == 'RUN'].describe().T

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, sharey=True)
sistemas[sistemas['Tipo_Execucao'] == 'FAST']['Duracao_Segundos'].plot(kind='hist', ax = ax)
sistemas[sistemas['Tipo_Execucao'] == 'RUN']['Duracao_Segundos'].plot(kind='hist', ax = ax2)

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, sharey=True)
sistemas[(sistemas['Tipo_Execucao'] == 'FAST') & (sistemas['Duracao_Segundos'] > 0)]['Duracao_Segundos'].plot(kind='hist', ax = ax)
sistemas[(sistemas['Tipo_Execucao'] == 'RUN') & (sistemas['Duracao_Segundos'] > 0)]['Duracao_Segundos'].plot(kind='hist', ax = ax2)

---
<span style="color: Red; font-size:1.5em">**Tratamento de dados**</span>

In [ ]:
#Nós temos duas execuções com tempos muito discrepantes das demais, iremos removê-los da análise
sorted(sistemas['Duracao_Segundos'], reverse=True)

In [ ]:
sistemas = sistemas[sistemas['Duracao_Segundos'] < 70000]

---

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, sharey=True)
sistemas[(sistemas['Tipo_Execucao'] == 'FAST') & (sistemas['Duracao_Segundos'] > 0)]['Duracao_Segundos'].plot(kind='hist', ax = ax, bins = 50)
sistemas[(sistemas['Tipo_Execucao'] == 'RUN') & (sistemas['Duracao_Segundos'] > 0)]['Duracao_Segundos'].plot(kind='hist', ax = ax2, bins = 50)

## 2. Análise de sistemas

In [ ]:
qtd_execs_sistemas = pd.DataFrame(sistemas.Sistema_Principal.value_counts())
qtd_execs_sistemas.reset_index(inplace=True)
qtd_execs_sistemas.rename(columns = {'index': 'sistema_principal', 'Sistema_Principal': 'qtd_total'}, inplace = True)

In [ ]:
qtd_execs_sistemas.head()

In [ ]:
qtd_execs_sistemas.shape

Temos, no total, **203 sistemas principais diferentes**. Vamos analisar o % de tempo zerado nós temos para cada um deles:

In [ ]:
qtd_execs_sistemas_zerados = pd.DataFrame(sistemas[sistemas['Duracao_Segundos'] == 0].Sistema_Principal.value_counts())
qtd_execs_sistemas_zerados.reset_index(inplace=True)
qtd_execs_sistemas_zerados.rename(columns = {'index': 'sistema_principal', 'Sistema_Principal': 'qtd_zerados'}, inplace = True)

In [ ]:
qtd_execs_sistemas_zerados.head()

In [ ]:
qtd_execs_sistemas = pd.merge(qtd_execs_sistemas, 
                              qtd_execs_sistemas_zerados, 
                              how='left', on='sistema_principal')

In [ ]:
qtd_execs_sistemas['%_zerados_total'] = qtd_execs_sistemas['qtd_zerados']/qtd_execs_sistemas['qtd_total']

In [ ]:
qtd_execs_sistemas.head()

In [ ]:
#Sistemas com mais de 80% de suas execuções tempos zerados
qtd_execs_sistemas[qtd_execs_sistemas['%_zerados_total'] > 0.8].shape

In [ ]:
qtd_execs_sistemas[qtd_execs_sistemas['%_zerados_total'] > 0.8]['qtd_total'].sum()

In [ ]:
#Sistemas com todas as execuções zeradas
qtd_execs_sistemas[qtd_execs_sistemas['%_zerados_total'] == 1].shape

Dos nossos 203 sistemas, **60 possuem mais de 80% de suas execuções zeradas, sendo que 31 deles estão totalmente zerados**

In [ ]:
sistemas_80_mais_zerados = qtd_execs_sistemas[qtd_execs_sistemas['%_zerados_total'] > 0.8]['sistema_principal'].tolist()
sistemas_total_zerados = qtd_execs_sistemas[qtd_execs_sistemas['%_zerados_total'] == 1]['sistema_principal'].tolist()

Para que nossa regressão da duração faça sentido, iremos retirar os sistemas que possuem mais de 80% de seu tempo zerado, removendo **4427** linhas do nosso dataset principal

---
<span style="color: Red; font-size:1.5em">**Tratamento de dados**</span>

In [ ]:
#O til é para indicar o contrário da condição, portanto que os demais sistemas que tenham tempo zerado menor ou igual a 80
sistemas = sistemas[~sistemas['Sistema_Principal'].isin(sistemas_80_mais_zerados)]

In [ ]:
sistemas.shape

---

## 3. Análise dos dados nulos

In [ ]:
sistemas.isna().sum()

Mais da metade de nossas observações em Motivo_Execucao estão nulas, portanto iremos remover essa feature, visto que ela pouco agregaria para nossa regressão

In [ ]:
sistemas.drop('Motivo_Execucao', axis=1, inplace = True)

In [ ]:
sistemas.isna().sum()

Ainda temos alguns valores nulos em target. Iremos substitui-lo por zero para indicar que eles não foram faturados

In [ ]:
sistemas.fillna(0, inplace = True)

In [ ]:
sistemas.isna().sum()

In [ ]:
etios sedan cinza zhl-9817

## 3. Hora de regredir

Primeiro, iremos separar os datasets, ficando apenas com aquelas execuções não zeradas para rodar nosso modelo

In [ ]:
lista_colunas = ['Sistema_Principal',
                 'ID_RUN',
                 'Classificacao',
                 'Duracao_Segundos',
                 'Qtde_StepsATTACHMENT',
                 'Qtde_TotalSteps',
                 'Tipo_Execucao',
                 'Target']

In [ ]:
sistemas_zerados = sistemas[sistemas['Duracao_Segundos'] == 0].copy()
sistemas_com_tempo = sistemas[sistemas['Duracao_Segundos'] != 0].copy()

In [ ]:
print('Observações zeradas: ' + str(sistemas_zerados.shape[0]))
print('Observações não zeradas: ' + str(sistemas_com_tempo.shape[0]))

Agora, criaremos um dataset de treino do nosso modelo e outro de teste para verificar a acurácia dele

## Modelo Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
#Primeiro, iremos fazer a regressão sem o nome dos sistemas e utilizando a qtd total de steps
X = sistemas_com_tempo[['Sistema_Principal',
                        'Classificacao',
                        'Qtde_TotalSteps',
                        'Tipo_Execucao',
                        'Target']]
X = pd.get_dummies(X, drop_first=True)
y = sistemas_com_tempo['Duracao_Segundos']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
modelo_RF = RandomForestRegressor()
modelo_RF = modelo_RF.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error

yhat_train = modelo_RF.predict(X_train) # previsao dos dados de treino para calcular as métricas
yhat_test = modelo_RF.predict(X_test) # previsao dos dados de teste para calcular métricas

print('-----Dados de Treino-----')
print('MSE - treino', mean_squared_error(y_train, yhat_train))
print('MAE - treino', median_absolute_error(y_train, yhat_train))
print('R2 - treino', r2_score(y_train, yhat_train))

print('\n-----Dados de Teste-----')
print('MSE - test', mean_squared_error(y_test, yhat_test))
print('MAE - test', median_absolute_error(y_test, yhat_test))
print('R2 - test', r2_score(y_test, yhat_test))

## Modelo Linear Regression

In [ ]:
X = sistemas_com_tempo[['Sistema_Principal',
                        'Classificacao',
                        'Qtde_TotalSteps',
                        'Tipo_Execucao',
                        'Target']]
X = pd.get_dummies(X, drop_first=True)
y = sistemas_com_tempo[['Duracao_Segundos']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
modelo_MQO = LinearRegression()
modelo_MQO = modelo_MQO.fit(X_train, y_train)

In [ ]:
yhat_train = modelo_MQO.predict(X_train) # previsao dos dados de treino para calcular as métricas
yhat_test = modelo_MQO.predict(X_test) # previsao dos dados de teste para calcular métricas

print('-----Dados de Treino-----')
print('MSE - treino', mean_squared_error(y_train, yhat_train))
print('MAE - treino', median_absolute_error(y_train, yhat_train))
print('R2 - treino', r2_score(y_train, yhat_train))

print('\n-----Dados de Teste-----')
print('MSE - test', mean_squared_error(y_test, yhat_test))
print('MAE - test', median_absolute_error(y_test, yhat_test))
print('R2 - test', r2_score(y_test, yhat_test))

## Modelo XGBoost